# Splice example
This notebook shows how the splice class can be used to concatenate multiple spectra from a different energy range together. 

In [1]:
%matplotlib qt

In [2]:
import pyEELSMODEL.api as em
import numpy as np
import os
import matplotlib.pyplot as plt

## Simulation
Three spectra will be simulated which have some overlapping regions and where the acquisition times are different (1, 10 and 1000 ms). The current is assumed to be constant meaning that the different acquisition times changes the amount of signal detected per spectrum. *This part part is not very important to understand how to utilize the splicing class*

In [3]:
def make_circular_mask(xco, yco, Rin, Rout, shape):
    XX, YY = np.meshgrid(np.arange(shape[1])-yco, np.arange(shape[0])-xco)
    
    R = np.sqrt(XX**2+YY**2)
    
    mask = np.zeros(shape)
    boolean = (R>=Rin) & (R<Rout)
    mask[boolean] = 1
    return mask

def make_rectangular_mask(xco, yco, width,height, shape):
    mask = np.zeros(shape)
    mask[xco:xco+width, yco:yco+height] = 1
    return mask

In [5]:
xsize = 128
ysize = 128

elements = ['C', 'N', 'O', 'Fe', 'Al']
edges = ['K', 'K', 'K', 'L', 'K']
Zs = [6, 7, 8, 26, 14]

maps = np.zeros((len(elements),xsize,ysize))


mask0 =make_rectangular_mask(5, 5, 20, 20, (xsize,ysize))
mask1 =  make_rectangular_mask(90, 90, 20, 30, (xsize,ysize))
mask2 = make_circular_mask(xsize//2, ysize//2, 20, 30, (xsize,ysize))
mask3 = make_circular_mask(xsize//2, ysize//2, 0, 20, (xsize,ysize))

maps[0] = 1
maps[1] = 2*mask0 + mask1
maps[2] = mask2
maps[3] = mask3+0.5*mask2
maps[4] = mask0 + mask1 +mask2 

In [10]:
#add some thickness variation in the eels spectra, not 
tlambda_map = np.zeros((xsize, ysize))
for ii in range(maps.shape[0]):
    tlambda_map += Zs[ii]*maps[ii]

tlambda_map = tlambda_map/tlambda_map.max()

In [13]:
settings = (300e3, 1e-9, 20e-3)
msh = em.MultiSpectrumshape(1, 150, 2048, xsize, ysize)

In [15]:
sim = em.CoreLossSimulator(msh, elements, edges, maps, tlambda_map, settings)
sim.bg_method='convolved'
sim.add_poisson = False #no poisson noise is added at this stage
sim.simulate_multispectrum()
hl = sim.multispectrum

16384it [00:12, 1281.85it/s]
16384it [00:35, 462.57it/s]

Multispectrum is simulated


In [16]:
spec1 = hl.get_interval((0.,500.))
spec2 = hl.get_interval((400.,900.))
spec3 = hl.get_interval((800.,2500.))

Define the acquisition times of the three different multispectra

In [17]:
specs = [spec1, spec2, spec3]
acq_times = np.array([1, 10, 1000])

In [18]:
for acq_time, spec in zip(acq_times, specs):
    spec.multidata = np.random.poisson(acq_time * spec.multidata).astype('float64') #needs to be flaot for multispectrum

## Splicing 
The different multispectrums defined in *specs* are being used to splice together. The acquisition times are needed to make a good average of the different spectra. This assumes that the count rate is constant during the acquisition of the three spectra. 

In [19]:
acq_times = [1, 10, 1000] 

In [20]:
#visualization
em.MultiSpectrumVisualizer(specs)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [21]:
from pyEELSMODEL.operators.splice import Splice

In [22]:
splice = Splice(specs, acq_times)
spliced_spectrum = splice.splice_multispectra()

16384it [00:09, 1713.01it/s]


The spliced multispectrum is visualized in the next cell. Note that the noise distribution has now changed and is not Poissonian anymore, the part with the longer acquisition time has significanlty lower noise as would expected from Poissonian. This could introduce some bias when doing elemental mapping since this assumes Poissonian noise. This can be solved by adding proper weights into the fitter but this is not implemented at this point

In [23]:
em.MultiSpectrumVisualizer([spliced_spectrum], logscale=False)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
